In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [2]:
raw_pdf = pd.read_csv('dataKCx.csv')

opening_time = pd.Timestamp('06:55:00').strftime("%H:%M:%S")
raw_pdf.loc[raw_pdf['tradingphase'] == 'OPENING AUCTION','time'] = opening_time
closing_time = pd.Timestamp('15:30:00').strftime("%H:%M:%S")
raw_pdf.loc[raw_pdf['tradingphase'] == 'CLOSING AUCTION','time'] = closing_time

c_pdf = raw_pdf[(raw_pdf['tradingphase'] == 'CONTINUOUS') | (raw_pdf['tradingphase'] == 'OPENING AUCTION')]


raw_pdf['date_time'] = pd.to_datetime(raw_pdf['date'] + 'T' + raw_pdf['time'])
raw_pdf.set_index('date_time')

,Unnamed: 0,date,time,volume,tradingphase
date_time,,,,,
2020-03-30 06:55:00,1,2020-03-30,06:55:00,466939.0,OPENING AUCTION
2020-03-30 07:00:00,2,2020-03-30,07:00:00,199240.0,CONTINUOUS
2020-03-30 07:05:00,3,2020-03-30,07:05:00,399928.0,CONTINUOUS
2020-03-30 07:10:00,4,2020-03-30,07:10:00,355929.0,CONTINUOUS
2020-03-30 07:15:00,5,2020-03-30,07:15:00,290592.0,CONTINUOUS
...,...,...,...,...,...
2023-03-08 16:10:00,68324,2023-03-08,16:10:00,589997.0,CONTINUOUS
2023-03-08 16:15:00,68325,2023-03-08,16:15:00,618924.0,CONTINUOUS
2023-03-08 16:20:00,68326,2023-03-08,16:20:00,733780.0,CONTINUOUS


In [29]:
#simple input: the last n_volumns (including present) are added.
size_data = len(raw_pdf['volume'])
training_split = 0.7
test_split = 0.2
validation_split = 1 - training_split - test_split
assert training_split + test_split + validation_split == 1, "Error, sum of training, test and validation datasets must be 1"
dates_pdf = raw_pdf['date'].drop_duplicates()
n_dates = len(dates_pdf)

n_volumns = 104*2
last_training_date_idx = math.floor(n_dates*training_split)
last_testing_date_idx = last_training_date_idx + math.floor(n_dates*test_split)
last_validation_date_idx = n_dates - 1
ts_length = n_volumns

training_pdf = raw_pdf[raw_pdf['date'].isin(dates_pdf[:last_training_date_idx + 1])]






,Unnamed: 0,date,time,volume,tradingphase,date_time
47835,47836,2022-04-26,15:10:00,552275.0,CONTINUOUS,2022-04-26 15:10:00
47836,47837,2022-04-26,15:15:00,608999.0,CONTINUOUS,2022-04-26 15:15:00
47837,47838,2022-04-26,15:20:00,870450.0,CONTINUOUS,2022-04-26 15:20:00
47838,47839,2022-04-26,15:25:00,823776.0,CONTINUOUS,2022-04-26 15:25:00
47839,47840,2022-04-26,15:30:00,28902231.0,CLOSING AUCTION,2022-04-26 15:30:00
